# Yijing Oracle System Tests

This notebook provides comprehensive testing of the Yijing Oracle implementation. We'll test:
1. Basic initialization and configuration
2. Hexagram generation and transformation
3. Single consultation mode
4. Dialogue consultation mode
5. Error handling and edge cases

In [ ]:
import os
import sys
from pathlib import Path
import logging
from typing import Dict, Any
from IPython.display import display, Markdown
from pydantic_settings import BaseSettings

from yijing import settings, LogLevel

In [ ]:
# Configure logging
logging.basicConfig(level=logging.INFO)

# Set debug mode for detailed output
settings.debug = True
settings.log_level = LogLevel.DEBUG

# Verify settings
print(f"Current log level: {settings.log_level}")
print(f"Debug mode: {settings.debug}")

In [ ]:

# Add project root to Python path if needed
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import all relevant modules
from yijing import (
    YijingOracle,
    ask_oracle,
    settings,
    ConsultationMode,
    LogLevel,
    Hypergram,
    HypergramLine,
    Hexagram,
    HexagramLine
)

## 1. Configuration and Setup

In [ ]:
def print_config():
    """Print current configuration settings"""
    print("Current Configuration:")
    print(f"Model Name: {settings.model_name}")
    print(f"Consultation Mode: {settings.consultation_mode}")
    print(f"Log Level: {settings.log_level}")
    print(f"Resources Directory: {settings.resources_dir}")
    print(f"Debug Mode: {settings.debug}")

print_config()

## 2. Test Hexagram Generation

In [ ]:
def test_hexagram_generation():
    """Test the generation and transformation of hexagrams"""
    # Create a test hypergram with known values
    test_lines = [
        HypergramLine(value=6),  # Changing Yin
        HypergramLine(value=7),  # Stable Yang
        HypergramLine(value=8),  # Stable Yin
        HypergramLine(value=9),  # Changing Yang
        HypergramLine(value=7),  # Stable Yang
        HypergramLine(value=8)   # Stable Yin
    ]
    
    hypergram = Hypergram(lines=test_lines)
    
    print("Test Hypergram:")
    for i, line in enumerate(hypergram.lines, 1):
        print(f"Line {i}: Value={line.value}, Type={line.line_type}")
    
    old_hex = hypergram.old_hexagram()
    new_hex = hypergram.new_hexagram()
    changing = hypergram.changing_lines()
    
    print("\nHexagram Analysis:")
    print(f"Old Hexagram: {old_hex.to_unicode_representation()}")
    print(f"New Hexagram: {new_hex.to_unicode_representation()}")
    print(f"Changing Lines: {changing}")

test_hexagram_generation()

## 3. Test Single Consultation Mode

In [ ]:
def test_single_consultation():
    """Test the oracle in single consultation mode"""
    settings.consultation_mode = ConsultationMode.SINGLE
    oracle = YijingOracle()
    
    question = "What opportunities should I focus on today?"
    response = oracle.get_response(question)
    
    print(f"Question: {question}\n")
    display(Markdown(response['answer']))
    
    print("\nHexagram Data:")
    hypergram_data = response['hypergram_data']
    print(f"Timestamp: {response['timestamp']}")
    print(f"Model Used: {response['model_used']}")
    print(f"Consultation Mode: {response['consultation_mode']}")

test_single_consultation()

## 4. Test Dialogue Consultation Mode

In [ ]:
def test_dialogue_consultation():
    """Test the oracle in dialogue consultation mode"""
    settings.consultation_mode = ConsultationMode.DIALOGUE
    oracle = YijingOracle()
    oracle.start_new_consultation()
    
    questions = [
        "What does my current situation look like?",
        "How should I interpret the changing lines?",
        "What advice does the new hexagram offer?"
    ]
    
    for i, question in enumerate(questions, 1):
        print(f"\nQuestion {i}: {question}\n")
        response = oracle.get_response(question)
        display(Markdown(response['answer']))
        print("\n---")

test_dialogue_consultation()

## 5. Error Handling Tests

In [ ]:
def test_error_handling():
    """Test error handling in various scenarios"""
    print("Testing invalid hexagram creation...")
    try:
        # Try to create a hexagram with too few lines
        invalid_lines = [HypergramLine(value=7) for _ in range(5)]
        Hypergram(lines=invalid_lines)
    except ValueError as e:
        print(f"Expected error caught: {e}")
    
    print("\nTesting invalid line values...")
    try:
        # Try to create a line with an invalid value
        HypergramLine(value=5)
    except ValueError as e:
        print(f"Expected error caught: {e}")

test_error_handling()

## 6. Performance Testing

In [ ]:
def test_performance():
    """Test performance of hexagram generation and consultation"""
    import time
    
    # Test hexagram generation performance
    start_time = time.time()
    iterations = 100
    
    for _ in range(iterations):
        hypergram_data = cast_hypergram()
    
    duration = time.time() - start_time
    print(f"Generated {iterations} hexagrams in {duration:.2f} seconds")
    print(f"Average time per hexagram: {(duration/iterations)*1000:.2f} ms")

test_performance()

## 7. Resource Loading Test

In [ ]:
def test_resource_loading():
    """Test loading of system resources"""
    try:
        resources = settings.load_resources()
        print("Resources loaded successfully!")
        
        for resource_name, content in resources.items():
            print(f"\n{resource_name}:")
            print(f"Content length: {len(content)} characters")
            print(f"Preview: {content[:100]}...")
            
    except Exception as e:
        print(f"Error loading resources: {e}")

test_resource_loading()